<img src="/resources/SFN_Logo_2018.png"> 

<h1 align="center">Population decoding with the Allen Brain Observatory</h1>

<h3 align="center">November 3, 2018</h3>

# This notebook:
## 1) Getting population responses from a brain observatory experiment 
## 2) Decoding a visual stimulus from the population activity


### First let's import things

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.titlesize'] = 16

from allensdk.core.brain_observatory_cache import BrainObservatoryCache

import numpy as np
import pandas as pd
import seaborn as sns
import os, sys, h5py

import matplotlib.pyplot as plt
%matplotlib notebook

drive_path = '/Volumes/Brain2018/visual_coding_2p/'
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

### What locations (visual areas, cre lines and imaging depths) are available?

In [ ]:
print(boc.get_all_targeted_structures())
print(boc.get_all_cre_lines())
print(boc.get_all_imaging_depths())

### What stimuli are available?

In [ ]:
print(boc.get_all_stimuli())

### Choose an experiment container

In [ ]:
# get an experiment from VISp / Cux2 / L2/3
# get raw responses and analysis object with events

visual_area = 'VISp'
cre_line ='Cux2-CreERT2'
imaging_depth = 275
exp_ind = 1

exps = boc.get_experiment_containers(targeted_structures=[visual_area], cre_lines=[cre_line], imaging_depths=[imaging_depth])
exp_cont = exps[exp_ind]

print('Number of experiment containers: '+str(len(exps)))
print(exps[0])

### Choose an experimental session

In [ ]:
exp_cont_id = exps[exp_ind]['id']
expt = boc.get_ophys_experiments(experiment_container_ids=[exp_cont_id], stimuli=['drifting_gratings'])[0]['id']

### Get the physiology data from that experiment.

In [ ]:
data_set = boc.get_ophys_experiment_data(ophys_experiment_id=expt)

In [ ]:
data_set.

In [ ]:
dff_t, dff = data_set.get_dff_traces()
events = boc.get_ophys_experiment_events(ophys_experiment_id=expt)

print(dff.shape)
print(events.shape)

In [ ]:
plt.figure();
plt.plot(dff_t, dff[0], label='Df/f'); plt.plot(dff_t, events[0], label='Events'); 
plt.xlabel('Time (s)'); plt.ylabel('Df/f'); plt.legend(frameon=False)

### How many neurons were segmented from this imaging experiment?

In [ ]:
N = dff.shape[0]
print(N)

In [ ]:
fig, ax = plt.subplots(1);
for i in range(10):
    ax.plot(dff_t, events[i]+i, color='k')
ax.set_xlabel('Time (s)'); ax.set_ylabel('Event size (df/f)');
ax.set_yticks([]); sns.despine(fig)
fig.tight_layout()

# What are the stimuli in this experiment?

In [ ]:
data_set.list_stimuli()

In [ ]:
stim_table = data_set.get_stimulus_table('drifting_gratings')

In [ ]:
print(type(stim_table))
print(stim_table[:10])

### Let's look at the population activity together with the stimulus

In [ ]:
fig, ax = plt.subplots(1);
for i in range(10):
    ax.plot(dff_t, events[i]+i, color='k')


cmap = matplotlib.cm.get_cmap('viridis')
norm = matplotlib.colors.Normalize(vmin=-45., vmax=315.)
normY = norm(stim_table['orientation'])

Nstim = len(stim_table)

for n in range(Nstim):
    start, end = stim_table['start'][n], stim_table['end'][n]
    plt.axvspan(xmin=dff_t[start], xmax=dff_t[end], color=cmap(normY[n]), alpha=0.5)

ax.set_xlabel('Time (s)'); ax.set_ylabel('Event size (df/f)');
ax.set_xlim((dff_t[stim_table['start'][0]], dff_t[stim_table['end'][20]]))
ax.set_yticks([]); sns.despine(fig)

# How can we relate the population activity to the stimulus?

<img src="decode_fig.png"> 

# Linear discriminant analysis (LDA): a probabilistic model of population responses

### Bayes' Rule tells us how to decompose joint probabilities: <br>

\begin{equation} \begin{aligned} 
\Large P(x, y) = P(y | x ) \, P(x) 
\end{aligned} \end{equation}

\begin{equation} \begin{aligned} 
\Large P(y | x ) = \frac{P(x|y) P(y)}{P(x)}
\end{aligned} \end{equation}

\begin{equation} \begin{aligned} 
\Large P(y | x ) = \frac{P(x|y) P(y)}{\sum_y P(x|y)P(y)}
\end{aligned} \end{equation}

### LDA corresponds to a Gaussian $P(x|y)$, along with the assumption that the covariance does not depend on $y$

# Let's get X!

In [ ]:
analysis_path = os.path.join(drive_path,'ophys_experiment_analysis')
analysis_file = os.path.join(analysis_path, str(expt)+'_three_session_A_analysis.h5')
mean_sweep_response = pd.read_hdf(analysis_file, 'analysis/mean_sweep_response_dg')

In [ ]:
print(mean_sweep_response.shape)

mean_sweep_response = np.array(mean_sweep_response)
plt.figure();
plt.plot(mean_sweep_response[:, 0], mean_sweep_response[:, 1], 'k.')
plt.xlabel('roi 1 response')
plt.ylabel('roi 0 response')

# The mean sweep response object is the trial-averaged df/f. Let's get it from events.

In [ ]:
def get_response_table(traces, stim_table, stim_type):

    '''
    param traces: units x time trace of activity
    param stim_table: dictionary of stimulus start / stop times and identities
    param stim_type: one of boc.get_all_stimuli()
    
    returns: numpy array of size (stimulus trials, units) with the summed activity in each trial
    '''
    
    Ncells = traces.shape[0]
    Nstim = len(stim_table)

    ind_start = stim_table.start.values
    ind_end = stim_table.end.values

    response_array = np.zeros((Nstim, Ncells))
    for i in range(Nstim):
        response_array[i] = np.mean(traces[:, ind_start[i]:ind_end[i]], axis=1)

    return response_array  # stim x cells

In [ ]:
mean_sweep_response_events = get_response_table(traces=events, stim_table=stim_table, stim_type='drifting_gratings')

X = np.array(mean_sweep_response_events)
print(X.shape)

In [ ]:
plt.figure();
plt.plot(mean_sweep_response[:, 0]/100., mean_sweep_response_events[:, 0], 'k.')

In [ ]:
stim_category = 'orientation'
Y = np.array(stim_table[stim_category]) # get stimulus labels
Y[~np.isfinite(Y)] = -45 # give blank stim a nicer label

print(Y[:10])
print(np.unique(Y))

In [ ]:
X = X[:, :2] # pick two neurons
print(X.shape)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis()
# LDA?

In [ ]:
LDA.fit(X, Y)

plt.figure()
plt.bar(.5, LDA.score(X, Y), width=0.2)
plt.ylabel('Training score'); plt.xlim((0, 1)); plt.ylim((0, 1.))

# How can we tell what points this trained classifier will classify as each direction?

In [ ]:
def decision_surface2d(x_plot, y_plot, classifier):
    
    '''
    param x: range for x axis
    param y: range for y axis
    param classifier: trained classifier with predict method
    '''
    
    xx, yy = np.meshgrid(x_plot, y_plot)
    X =(np.c_[xx.ravel(), yy.ravel()])
    
    return classifier.predict(X).reshape(xx.shape)

In [ ]:
x_plot = np.linspace(-1, 1, 100)

plt.figure();
plt.imshow(decision_surface2d(x_plot, x_plot, LDA), interpolation='none', extent=(-1, 1, 1, -1), clim=(-45, 315));
plt.xlabel('possible roi 1 response'); plt.ylabel('possible roi 0 response')
cbar = plt.colorbar(); plt.text(1.6, 0, 'Orientation', rotation=90, verticalalignment='center');
cbar.set_ticks(np.unique(Y))
cbar.set_ticklabels(['blank']+np.unique(Y)[1:].astype('int').tolist() )


# How sensitive is the decision surface to the amount of training data?

In [ ]:
num_points = range(10, 12)

x_min = -1.5 * np.amax(X[:num_points[-1]])
x_plot = np.linspace(x_min, -1*x_min, 100)

Nplots = len(num_points)
if Nplots % 2 != 0: Nplots += 1

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

for n, ax in enumerate(fig.axes):
    
    num = num_points[n]
    LDA.fit(X[:num], Y[:num])
    
    im = ax.imshow(decision_surface2d(x_plot, x_plot, LDA), interpolation='none', extent=(x_min, -1*x_min, -1*x_min, x_min), clim=(-45, 315))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.plot(X[:num, 0], X[:num, 1], 'k.')
    ax.plot(0, 0, 'r.')
    ax.set_title(str(num)+' points', fontsize=16)

plt.subplots_adjust(hspace=0.4)
fig.text(.5, .01, 'roi 1 response', horizontalalignment='center')
fig.text(.05, .5, 'roi 0 response', rotation=90, verticalalignment='center')

cbar = fig.colorbar(im, ax=fig.axes)
fig.text(.87, .5, 'Orientation', rotation=90, verticalalignment='center')
cbar.set_ticks(np.unique(Y))
cbar.set_ticklabels(['blank']+np.unique(Y)[1:].astype('int').tolist() )


In [ ]:
print(Y[:10])
print(Y[:11])
print(X[10])

In [ ]:
num_points = range(10, 16)

x_min = -1.5 * np.amax(X[:num_points[-1]])
x_plot = np.linspace(x_min, -1*x_min, 100)
                     
Nplots = len(num_points)
if Nplots % 2 != 0: Nplots += 1

fig, ax = plt.subplots(2, Nplots/2, figsize=(10, 6))

for n, ax in enumerate(fig.axes):
    
    num = num_points[n]
    LDA.fit(X[:num], Y[:num])
    
    im = ax.imshow(decision_surface2d(x_plot, x_plot, LDA), interpolation='none', extent=(x_min, -1*x_min, -1*x_min, x_min), clim=(-45, 315))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.plot(X[:num, 0], X[:num, 1], 'k.')
    ax.plot(0, 0, 'r.')
    ax.set_title(str(num)+' points', fontsize=16)

plt.subplots_adjust(hspace=0.4)
fig.text(.5, .01, 'roi 1 response', horizontalalignment='center')
fig.text(.05, .5, 'roi 0 response', rotation=90, verticalalignment='center')

cbar = fig.colorbar(im, ax=fig.axes)
cbar.set_ticks(np.unique(Y))
cbar.set_ticklabels(['blank']+np.unique(Y)[1:].astype('int').tolist() )
fig.text(.87, .5, 'Orientation', rotation=90, verticalalignment='center')


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">
<p>In order to avoid over-fitting, we should evaluate our classifier on different data than we used to train it. This is called cross-validation. If we have a data set of K points, we can hold out a fraction of our data to use as testing data. The classifier's ability to generalize shows us how it can learn the class distributions, rather than the training data.

In [ ]:
from sklearn.model_selection import KFold, LeaveOneOut, StratifiedKFold
# check sklearn.cross_validation if your version doesn't have model_selection

n_splits = 2
kf = StratifiedKFold(n_splits=n_splits)
kf?

In [ ]:
n = 0
fig, ax = plt.subplots(1, 2, figsize=(9, 4), sharex=True, sharey=True)

for train, test in kf.split(X, Y):
    ax[0].plot(X[train, 0], X[train, 1], '.', label='fold '+str(n))
    ax[1].plot(X[test, 0], X[test, 1], '.', label='fold '+str(n))
    n += 1
    
ax[0].set_title('Train')
ax[1].set_title('Test')
ax[0].legend(loc=0, frameon=False)
fig.text(.5, .01, 'roi 1 response', horizontalalignment='center')
fig.text(.01, .5, 'roi 0 response', rotation=90, verticalalignment='center')
fig.tight_layout()

In [ ]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits)

n = 0
fig, ax = plt.subplots(1)

for train, test in kf.split(X, Y):
    LDA.fit(X[train], Y[train])
    
    if n == 0:
        ax.bar(n-.25, LDA.score(X[train], Y[train]), facecolor='k', width=0.5, label='Train')
        ax.bar(n+.25, LDA.score(X[test], Y[test]), facecolor='r', width=0.5, label='Test')
    else:
        ax.bar(n-.25, LDA.score(X[train], Y[train]), facecolor='k', width=0.5)
        ax.bar(n+.25, LDA.score(X[test], Y[test]), facecolor='r', width=0.5)

    n += 2

ax.set_xticks(range(0, n_splits*2, 2))
ax.set_xticklabels(range(0, n_splits))
ax.set_xlabel('Fold')
ax.set_ylabel('Score')
ax.set_ylim((0, .3))
ax.legend(loc='upper left', frameon=False)
fig.tight_layout()

# How do the scores evolve as we use more training data?

In [ ]:
from sklearn.model_selection import learning_curve

num_total = X.shape[0]
train_sizes, train_scores, test_scores = learning_curve(LDA, X, Y, cv=5, train_sizes=np.arange(.1, 1.1, .1));

plt.figure()
plt.plot(train_sizes, train_scores, 'b');
plt.plot(train_sizes, test_scores, 'r');
plt.xlabel('Training points'); plt.ylabel('Score'); #plt.legend(loc=0, frameon=False);
plt.text(100, .6, 'Train', color='b')
plt.text(100, .5, 'Test', color='r')